In [4]:
print("----\n")

[MyPython] Info:The process :h:\Jupyter\Myjupyter-kernel\kernel\Vala\tmptpispml3.py
[MyPython] Info:python3 h:\Jupyter\Myjupyter-kernel\kernel\Vala\tmptpispml3.py 


----


[MyPython] Info:The process end:16020


In [5]:
##%overwritefile
##%file:src/compile_with_valac.py
##%noruncode
    def compile_with_valac(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-Wdeclaration-after-statement', '-Wvla', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=iso9899:199409', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c11', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        outfile=binary_filename
        # if self.linkMaths:
        #     cflags = cflags + ['-lm']
        # if self.wError:
        #     cflags = cflags + ['-Werror']
        # if self.wAll:
        #     cflags = cflags + ['-Wall']
        # if self.readOnlyFileSystem:
            # cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        # if self.bufferedOutput:
            # cflags = ['-DBUFFERED_OUTPUT'] + cflags

        for s in cflags:
            if s.startswith('-o'):
                if(len(s)>2):
                    outfile=s[2:]
                else:
                    outfile=cflags[cflags.index('-o')+1]
                    if outfile.startswith('-'):
                        outfile=binary_filename
                del cflags[cflags.index('--outFile')+1]
                del cflags[cflags.index('--outFile')]
                    
            binary_filename=outfile
        args = ['valac', source_filename] + ['-o', binary_filename]+ cflags  + ldflags
        self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename,args


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Vala\src/compile_with_valac.py created successfully


In [4]:
##%overwritefile
##%file:src/c_exec_valac_.py
##%noruncode
    def _exec_valac_(self,source_filename,magics):
        self._write_to_stdout('Generating executable file\n')
        with self.new_temp_file(suffix='.out') as binary_file:
            p,outfile,gcccmd = self.compile_with_valac(
                source_filename, 
                binary_file.name,
                self.get_magicsSvalue(magics,'cflags'),
                self.get_magicsSvalue(magics,'ldflags'),
                self.get_magicsbykey(magics,'env')
                )
            returncode=p.wait_end(magics)
            p.write_contents()
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if returncode != 0:  # Compilation failed
                self._log(''.join((str(s) for s in gcccmd))+"\n",3)
                self._log("valac exited with code {}, the executable will not be executed".format(returncode),3)

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\Vala\src/c_exec_valac_.py created successfully
